In [41]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [42]:
lottery_dataframe = pd.read_csv("lottery.csv", sep=",")
lottery_dataframe = lottery_dataframe.reindex(np.random.permutation(lottery_dataframe.index))

In [43]:
# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 100
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_classes = 100 # total classes (00-99)

In [44]:
# Choose the first 578 (out of 684) examples for training.
train_data = lottery_dataframe.head(580).values[:,:3].astype(np.float32)
train_labels = lottery_dataframe.head(580).values[:,6].astype(np.float32)

# Choose the last 106 (out of 684) examples for validation.
valid_data = lottery_dataframe.tail(100).values[:,:3].astype(np.float32)
valid_labels = lottery_dataframe.tail(100).values[:,6].astype(np.float32)

In [45]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': train_data}, y=train_labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)

In [46]:
# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['images']
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer

In [47]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    logits = neural_net(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [48]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

In [49]:
# Train the Model
model.train(input_fn, steps=num_steps)

ERROR:tensorflow:Model diverged with loss = NaN.


NanLossDuringTrainingError: NaN loss during training.

In [ ]:
#
# TODO test features
# TODO test labels
#
# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': valid_data}, y=valid_labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn)